### **Итоговое задание Соколова Александра**  
### по Проекту 5. Car Price prediction 
####  Юнит 6. Основные алгоритмы машинного обучения. Часть II (отредактирован 19.9.2020)
---
## Часть 2. Модель
### 1. Импорт библиотек

In [ ]:
import pandas as pd
import pandas_profiling
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.feature_selection import f_classif, mutual_info_classif
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import ExtraTreeRegressor
from tqdm import tqdm
from itertools import combinations
from scipy.stats import ttest_ind
from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
pd.options.mode.chained_assignment = None

import os

# этот блок закомментирован так как используется только на kaggle
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
PATH_to_file = '/kaggle/input/sf-dst-car-price/'
PATH_to_file_data = '/kaggle/input/parsing-all-moscow-auto-ru-09-09-2020/'

# # # этот блок закомментирован так как используется только локальной машине
# from importlib import reload
# print(os.listdir('./data'))
# PATH_to_file = './data/'

In [ ]:
import utils_module09092020 as utils

In [ ]:
RANDOM_SEED = 42
!pip freeze > requirements.txt
CURRENT_DATE = pd.to_datetime('19/09/2020')

### 2. Импорт данных

In [ ]:
df_train = pd.read_csv(PATH_to_file_data +'all_auto_ru_09_09_2020.csv')
df_test = pd.read_csv(PATH_to_file+'test.csv')
df_submit = pd.read_csv(PATH_to_file+'sample_submission.csv')
pd.set_option('display.max_columns', None)
print('Размерность тренировочного датасета: ', df_train.shape)
display(df_train.head(2))
print('Размерность тестового датасета: ', df_test.shape)
display(df_test.head(2))
print('Размерность датасета c примером сабмишена: ', df_submit.shape)
display(df_submit.head(2))

### 2. Проверка и доработка датасета после парсинга для слияния с тестом

In [ ]:
# сравним датасеты 
utils.check_df_before_merg(df_train, df_test)

In [ ]:
# починим датасет перед использованием и приведем его к виду теста

# на первый взгляд может, что перевод во float достаточно бесмыссленный шаг ведь кол-во дверей автомобиля это целое число. Но напомню что мы сейчас на этапе приведения датасета после парсинга к виду тестового. На этапе анализа датасета мы сможем более детально посмотреть что происходит в тестовом датасете (вероятно там есть ошибки которые надо исправить и мы можем потерять эту информацию не знаю сколько ошибок и какие они) 

# начнем с int64!=float64 и приведем все к float 
list_cols_to_repair = ['productionDate', 'mileage']
for col in list_cols_to_repair:
        df_train[col] = df_train[col].astype('float64')

# теперь починим int64!=float64 и приведем все к object
list_cols_to_repair = ['enginePower', 'Состояние', 'Владельцы']
for col in list_cols_to_repair:
        df_train[col] = df_train[col].astype('object')

# теперь починим bool!=object
# предварительно райдем в датасет и убедимся, что True соответсвует Растаможен зайдя на сайте (этот код тут не приводится, чтобы не нагружать ноутбук)
df_train['Таможня'] = df_train['Таможня'].map({True: 'Растаможен', False:'Не растаможен'})

In [ ]:
utils.check_df_before_merg(df_train, df_test)

Резюме - несоответствие типов осталось в 5 столбцах \[price, start_date, hidden, model, id\], в том числе price - целевая переменная которую мы будем моделировать, три столбца чтобы попробовать обогатить данные, и столбцец id, который нам будет нужен для формирования файла submition.  
Теперь обратим внимание на значения и приведем их к общему виду 

In [ ]:
# bodyType - надо привести значения к нижнему регистру (оставим закадром, проверку уникальных значений трейна и теста, чтобы прийти к такому выводу)
df_train['bodyType'] = df_train['bodyType'].apply(lambda x: str(x).lower())
utils.nunique_not_found(df_train, df_test, 'bodyType')


# color - цвета спарсились в шестнадцатеричном коде цветов. Но их всего 16 поэтому воспользуемся сайтом (https://hysy.org/color) и сделаем словарь цветов. (оставим закадром, проверку уникальных значений трейна и теста, чтобы сделать соответсвия цветов) 
dict_color = {'040001':'чёрный', 'EE1D19':'красный', '0000CC':'синий', 
              'CACECB':'серебристый', '007F00':'зелёный', 'FAFBFB':'белый', 
              '97948F':'серый', '22A0F8':'голубой', '660099':'пурпурный', 
              '200204':'коричневый', 'C49648':'бежевый', 'DEA522':'золотистый', 
              '4A2197':'фиолетовый', 'FFD600':'жёлтый', 'FF8649':'оранжевый', 
              'FFC0CB':'розовый'}
df_train['color'] = df_train['color'].map(dict_color)
utils.nunique_not_found(df_train, df_test, 'color')

# vehicleTransmission - тип трасмиссии по словарю из 4 значений
df_train['vehicleTransmission'] = df_train['vehicleTransmission'].map({'MECHANICAL':'механическая', 'AUTOMATIC':'автоматическая', 'ROBOT':'роботизированная', 'VARIATOR':'вариатор'})


# Руль - ну тут все просто - Правый или Левый в нижнем регистре с большой буквы
df_train['Руль'] = df_train['Руль'].map({'RIGHT':'Правый', 'LEFT':'Левый'})
utils.nunique_not_found(df_train, df_test, 'Руль')

# ПТС - тут все просто - Оригинал или Дубликат в нижнем регистре с большой буквы
df_train['ПТС'] = df_train['ПТС'].map({'ORIGINAL':'Оригинал', 'DUPLICATE':'Дубликат'})
utils.nunique_not_found(df_train, df_test, 'ПТС')

# Владельцы - Оригинал или Дубликат в нижнем регистре с большой буквы
df_train['Владельцы'] = df_train['Владельцы'].map({3.0:'3 или более', 2.0:'2\xa0владельца', 1.0:'1\xa0владелец'})
utils.nunique_not_found(df_train, df_test, 'Владельцы')

# Владение - в таблице сверху в первых строках Nan поэтому нужно отдельно проверить
# оказалось что в тесте очень сложная строковая запись тогда как в трейне в значениях более структурируемые словари (УчТЕМ этот момент и разберем этот столбец на этапе анализа, вероятно будет сразу полезно создать дополнительные временные фичи) 
utils.nunique_not_found(df_train, df_test, 'Владение');

Резюме - несоответсвие значений в одинаковых типах осталось только в одном столбце - Владение, но приведение к единому виду целеообразно оставить до детально анализа по переменным, так как там зашиты данные о годе и месяце владения и в процессе обработке может быть сразу целесообразно будет создавать новые временные фичи.  

### 3. Слияние датасетов в один

In [ ]:
# ВАЖНО! для корректной обработки признаков объединяем трейн и тест в один датасет
df_train['Train'] = 1 # помечаем где у нас трейн
df_test['Train'] = 0 # помечаем где у нас тест

df = df_train.append(df_test, sort=False).reset_index(drop=True) # объединяем
#!Обратите внимание объединение датасетов является потенциальной опасностью для даталиков

In [ ]:
# проверка  после слияния
df.head(2)

### 4. Предварительный анализ данных

In [ ]:
# анализ тренировочной части
pandas_profiling.ProfileReport(df[df['Train']==1])

In [ ]:
# анализ тестовой части закомментирован так как возникает ошибка переполнения RAM на kaggle
# # анализ тестовой части
# pandas_profiling.ProfileReport(df[df['Train']==0])

In [ ]:
# выведем сводную информацию по датасету df без теста kaggle
utils.describe_without_plots_all_collumns(df[df['Train']==1], short=True)

In [ ]:
# выведем сводную информацию по тесту
utils.describe_without_plots_all_collumns(df[df['Train']==0], short=True)

Резюме по предварительному анализу: (выводы идут по порядку выявления, а не по значимости)  
В тренировочной выборке 88968 строк, в тестовой 3837. Всего данных - 93215 строк. Всего 28 признаков, из них:
- 23 в тесте: 
  - 4 - числовых - mileage, modelDate, productionDate (один из них id - сервисный для генерации submition), 
  - 19 - категориальных (пока так, во время анализа, вероятно, какие-нибудь из них станут числовыми, например можность двигателя)
- 4 были добавлены во время парсинга: 1- числовой - price, 1 - временной ряд - start_date, 1 - бинарный - hidden, 1 - категориальный - model
- 1 - бинарный Train  - сервисный для разделения train и test
  
Детальная информация по трейну и тесту раздельно:  
- признак Таможня бесполезен так как содержит одно значение - Растаможен => удалим
- в трейне 2114 (2.4%) дубликатов по строкам => надо удалить
- высокая корреляция между modelDate и productionDate => надо еще внимательно посмотреть на этапе анализа перед тем как удалить
- пропуски в трейне:
  - description - 3254 (3.6%) => надо заполнить
  - Состояние - 89378 (100.0%) - мы не смогли во время парсинга раздобыть этот критерий => надо проанализировать заполненность этого признака в тесте и постараться заполнить по похожим критериям авто требующих ремонта - так как это влияет на цену 
  - Владельцы - 13588 (15.2%) => надо заполнить 
  - Владение - 60177 (67.3%) => очень много пропусков надо попробовать заполнить
  - hidden - 89378 (100.0%) => надо удалить
  - id - 89378 (100.0%) - это столбец для генерации файла submition => ничего не делаем
- пропуски в тесте:
  - Владение - 2084 (54.3%) => надо заполнить
  - price, start_date, hidden, model - 3837 (100.0%) - дополнительные фичи => надо проанализировать и использовать для генерации новых фич или удалить
- между vehicleConfiguration и еще тремя признаками: vehicleTransmission, engineDisplacement, numberOfDoors - высокая корреляция => надо проверить это при анализе и при необходимости удалить
- в тесте:
  - brand - только одно значение - "BMW" => надо дополнительно собрать информацию с обзоров цен на BMW и постараться оставить для анализа автомобили со схожими характеристиками, которые влюяют на цену
  - Руль - только одно значение - "Левый" => надо взять из трейна только авто с Левым рулем
  - Состояние - только одно значение - "Не требует ремонта" => надо обратить внимание на выбросы снизу по цене из трейна, чтобы не брать в учет авто требующие ремонта

Описания полей датасета
- bodyType - тип кузова автомобиля (https://ru.wikipedia.org/wiki/Типы_автомобильных_кузовов)
- brand - марка (бренд) автомобиля
- color	- цвет автомобиля (16 цветов: чёрный, красный, синий, серебристый, зелёный, белый, серый, голубой, пурпурный, коричневый, золотистый, фиолетовый, жёлтый, оранжевый, розовый)
- fuelType - тип топлива (5 видов топлива: бензин, дизель, электро, гибрид, газ)
- modelDate - дата выпуска модели (не путать с датой производства)
- name - комбинация нескольких характеристик (модели, объема двигателя и привода)
- numberOfDoors - кол-во дверей
- productionDate - дата производства авто
- vehicleConfiguration - комбинация типа трансмиссии, объема двигателя и мощности двигателя
- vehicleTransmission - тип трансмиссии (коробки передач) (4 вида: автоматическая, механическая, роботизированная, вариатор)
- engineDisplacement - объем двигателя в литрах
- enginePower - мощность двигателя (л.с.)
- description - дополнительные характеристики по комплектации авто
- mileage - пробег автомобиля (км.) - представляет собой общее количество километров, которое это транспортное средство проехало по дорогам с момента своего схода с конвейера производителя. 
- Комплектация - дополнительные характеристики по комплектации авто
- Привод - характеристика привода автомобиля, которое передает энергию от двигателя на колеса. Переднеприводные автомобили получают всю энергию двигателя на передние колеса. При заднем приводе энергия двигателя целиком поступает на задние колеса. Когда энергия двигателя передается на все четыре колеса автомобиля, такой привод является полным ('полный', 'передний', 'задний')
- Руль - характеристика руля по стороне расположения руля в автомобиле (Левый, Правый)
- Состояние - характеристика (необходимости проведения ремонта - требует ремонта или не требует ремонта)
- Владельцы - кол-во владельцев (3 значения: 1 владелец, 2 владельца, 3 и более)
- ПТС - тип ПТС (паспорт технического средства)(2 значения - оригинал, дубликат)
- Таможня - информация о том расстаможен автомобиль или нет (2 значения - Растаможен, Не ратаможен)
- Владение - срок владения автомобилем в годах и месяцах
- price - цена указанная в объявлении на авто.ру
- start_date - дата размещения объявления на авто.ру
- hidden - характеристика объявления на авто.ру
- model - название модели автомобиля (не путать с маркой (брендом), например: бренд или марка - BMW, модель - M6)
- Train - сервисный для разделения train и test
- id - - сервисный для генерации submition

In [ ]:
# внесем данные по типам переменных из резюме в списки 
# временной ряд (1)
time_cols = ['start_date']
# бинарная переменная (0) (hidden - не включаем так как решили удалить) итого  (0+1=1)
bin_cols = []
# категориальные переменные (19), ('Таможня' - не включаем  так как решили удалить) (19+1=20)
cat_cols = ['bodyType', 'brand', 'color', 'fuelType', 'name',
       'numberOfDoors', 'vehicleConfiguration',
       'vehicleTransmission', 'engineDisplacement', 'enginePower',
       'description', 'Комплектация', 'Привод', 'Руль', 'Состояние',
       'Владельцы', 'ПТС', 'Владение', 'model']
# числовые переменные (3) 
num_cols = ['mileage', 'modelDate', 'productionDate']
# сервисные переменные (2)
servis_cols = ['Train', 'id']
# целевая переменная (1)
target_col = ['price']
# итого 1+1+20+3+2+1=28

all_cols =cat_cols+num_cols+time_cols+servis_cols+bin_cols+ target_col
print(f'Кол-во столбцов, для дальнейшей работы после предварительного анализа:= {len(all_cols)}')

In [ ]:
# реализуем выводы из резюме перед детальным анализом по переменным
# берем только левый руль и тольк 26 столбцов
df = df.loc[df['Руль'] == 'Левый', all_cols]
# исключаем Руль из списка признаков для анализа итого остается 25 признаков из которых 2 сервисных (нужно проанализировать 23=25-2)
cat_cols.remove('Руль')
# удаляем дубликаты
df = df.drop_duplicates()
old_len_train = len(df[df['Train']==1])
print('Кол-во строк в трейне:= ', old_len_train)

### 5. Сбор дополнительной информации о BMW
---
Какие то важные факты для дальнейшего анализа с дополнениями и выводами:
- статистика по ценам обычно приводится по моделям, а в тесте признака модель нет => надо попробовать его восстановить
- существует общепринятое деление по классам - \[малый средний класс, средний класс, бизнес класс, представительский класс, внедорожники, купе, кабриолеты и родстеры\] => надо проверить возможно такая группировка позволит найти какие-то закономерности внутри классов
- обычно эта марка не включается в какие-то программы поддержки продаж для диллеров (типа сниженный процент от государства и т.п.)
- в России автомобили BMW собираются в Калининграде («Автотор») (3 серия, 5 серия, 7 серия, X3, X5, X1, X6) => соотвественно нет необходимости расстамаживать эти марки автомобилей
- увеличенный на 22 мм клиренс, более жёсткие амортизаторы и усиленные стабилизаторы, адаптированная под морозы электроника, сборка менее чувствительна к качеству бензина - это основные отличия русской сборки => вероятно диллеры учитывают ньансы для продажи авто в России, надо обратить внимание что входит в комплектацию, возможно есть вещи, которые существенно увеличивают стоимость
- ранее 3-4 летние авто сильно теряли в цене, теперь этот порог сдвинулся до 5-6 лет => вероятно это из-за расширения сроков гарантии на авто, когда подходят гарантия или пониженный по стоимости и более долгий техосмотр, владельцы продают автомобили
- на рынке этой марки в отличии от других присутсвует некоторая философия БМВ - в новом авто предположительно очень большая доля так называемых транзакционных издержек (оверхедов) + стоимость бренда, которые после покупки авто первым владельцем "сильно" снижаются, дальше для рынка остается себестоимость машины. Эта себестоимость торгуется на рынке, но BMW находит своих приверженцев, которые также платят чуть дороже, но за BMW, за свой стиль жизни, комфорт, за свою философию BMW => надо проверить действительно ли это так, сделать несколько срезов по маркам и моделям и оставить для анализа либо только БМВ либо авто, которые попадают в конкурентные с БМВ ниши.  

Использованы материалы с ресурсов:
- https://auto.ru/stats/cars/bmw/
- https://ru.wikipedia.org/wiki/BMW
- https://zen.yandex.ru/media/id/5987885f1410c361e4ba0f9c/bmv-nemeckoi-sborki-v-chem-otlichie-ot-kaliningradskih-mashin-5d4da573ec575b00ad872a7a
- https://zen.yandex.ru/media/pitstop/chto-oznachaiut-bukvy-i-cifry-v-nazvaniiah-bmw-5b16229497b07500a945be0a  
и другие

### 6. Детальный анализ по переменным
---
### 6.1 bodyType 
начнем с типа кузова bodyType, чтобы убедиться в том что значения по всем маркам и моделям примерно одинаковые или наоборот и тогда убрать нерелевантные типы кузова

In [ ]:
# так как признаков много создаем список проанализированных признаков, чтобы можно было посмотреть, что осталось сделать  
EDA_done_cols=[]

In [ ]:
temp_df_Train = df[df['Train']==1]
temp_df_Test = df[df['Train']==0]
print(f'Всего в тесте типов кузова:= {temp_df_Test.bodyType.nunique()}')
print(f'Всего в трейне типов кузова:= {temp_df_Train.bodyType.nunique()}')

In [ ]:
# посмотрим на них 
print(f'Список 11 типов кузова БМВ из теста:= {temp_df_Test.bodyType.unique()}')
print(f'Список первых 20 из 144 типов кузова из трейна:= {temp_df_Train.bodyType.unique()[:20]}')

In [ ]:
# оставляем только типы кузова как в тесте
list_bodyType_test = list(temp_df_Test.bodyType.unique())
df = df[df['bodyType'].isin(list_bodyType_test)]

***Резюме по bodyType:-*** Категориальный признак, оставили 11 типов кузовов, так как типы кузовов отличающиеся от кузовов в тесте очень специфичны и очень сильно влияют на класс автомобиля по которому по общим закономерностям формируется цена, оставим только типы кузова из теста, а потом после EDA проверим достаточно ли большая выборка трейна остается для моделирования или необходимо парсить другие сайты. Пропусков нет.

In [ ]:
old_len_train, EDA_done_cols = utils.result_EDA_feature('bodyType', df[df['Train']==1], df[df['Train']==0], 23, EDA_done_cols, old_len_train)

### 6.2 enginePower
забежим вперед и обработаем мощность двигателя потому что она нам потребуется для анализа марок

In [ ]:
# вспомним что по результатам предварительного анализа эта переменная была отнесена к категориальнымu, посмотрим почему
temp_df_Train = df[df['Train']==1]
temp_df_Test = df[df['Train']==0]
display(temp_df_Train.enginePower[:5])
display(temp_df_Test.enginePower[:5])

In [ ]:
# проверим все значения теста по мощности 
temp_df_Test.enginePower.unique()

In [ ]:
# в тесте вытащим значение мощности перед строкой ' N12'
df.loc[df['Train']==0, 'enginePower'] = df[df['Train']==0]['enginePower'].apply(lambda x: int(x.split()[0]))

In [ ]:
# переводим в int64
df['enginePower'] = df['enginePower'].astype('int64')

# перенесем признак в числовые
cat_cols.remove('enginePower')
num_cols.append('enginePower')

In [ ]:
utils.describe_without_plots('enginePower', df[df['Train']==1].enginePower)

In [ ]:
# очень странное минимальное значение можности, давайте посмотрим минимум мощности на тесте 
utils.describe_without_plots('enginePower', df[df['Train']==0].enginePower)

In [ ]:
df[(df['Train']==1) & (df['enginePower']<90)]['brand'].value_counts().plot(kind = 'barh', title='Cтатистика по маркам автомобилей с мощностью менее 90 л.с.')

In [ ]:
df[(df['Train']==1) & (df['enginePower']>625)]['brand'].value_counts().plot(kind = 'barh', title='Cтатистика по маркам автомобилей с мощностью более 625 л.с.')

In [ ]:
# удалим значения мощностей меньше 90 и выше 625
df=df[(df['enginePower']<=625) & (df['enginePower']>=90)]

In [ ]:
utils.four_plot_with_log2('enginePower', df[df['Train']==1])

***Резюме по enginePower:-*** обработали строки и перевели мощность в int64. Пропусков нет. Было какое-то небольшое значение мощности в трейне равное 11. Через это вышли на сравнение мощности с тестом и обрезали трейн по мощности двигателя то теста (более 90 и менее 625). О характере распределения и выбросах говорить пока рано, так как предстоит фильтрация по бренду, целесообразно вернуться к более детальному анализу после того как датасет устаканится. 

In [ ]:
# записываем признак в список проанализированных признаков
old_len_train, EDA_done_cols = utils.result_EDA_feature('enginePower', df[df['Train']==1], df[df['Train']==0], 23, EDA_done_cols, old_len_train)

### 6.3 brand
теперь посмотрим на марки - brand

In [ ]:
print(f'Всего в датасета марок:= {df.brand.nunique()}')

Вероятно не все из них нам подходят, сделаем сводные таблицы и посмотрим на средние значения цен по автомобилям по годам

In [ ]:
list_brand0 = utils.hbar_group_pivot_table(list_bodyType_test[0], 'price', df[df['Train']==1], 2013, 2020, 1.1)

In [ ]:
list_brand1 = utils.hbar_group_pivot_table(list_bodyType_test[1], 'price', df[df['Train']==1], 2015, 2019, 1.1)

In [ ]:
list_brand2 = utils.hbar_group_pivot_table(list_bodyType_test[2], 'price', df[df['Train']==1], 2000, 2010, 1.1)

In [ ]:
list_brand3 = utils.hbar_group_pivot_table(list_bodyType_test[3], 'price', df[df['Train']==1], 2015, 2019, 1.1)

In [ ]:
list_brand4 = utils.hbar_group_pivot_table(list_bodyType_test[4], 'price', df[df['Train']==1], 2015, 2019, 1.1)

In [ ]:
list_brand5 = utils.hbar_group_pivot_table(list_bodyType_test[5], 'price', df[df['Train']==1], 1990, 2010, 1.1)

In [ ]:
list_brand7 = utils.hbar_group_pivot_table(list_bodyType_test[7], 'price', df[df['Train']==1], 2000, 2005, 1.1)

In [ ]:
list_brand8 = utils.hbar_group_pivot_table(list_bodyType_test[8], 'price', df[df['Train']==1], 2000, 2010, 1.1)

In [ ]:
list_brand10 = utils.hbar_group_pivot_table(list_bodyType_test[10], 'price', df[df['Train']==1], 2015, 2019, 1.1)

In [ ]:
# список всех релевантных моделей
list_final_brand=list(set(list_brand0+list_brand1+list_brand2+list_brand3+list_brand4+list_brand5+list_brand7+list_brand8+list_brand10))
print('Список релевантных брендов авто:\n',*list_final_brand, '\n, их кол-во:=', len(list_final_brand), 'из 36')
print()

In [ ]:
# кроме этого в трейне нет некоторых авто БМВ с типом кузова
print('В трейне не оказалось авто БМВ с типом кузова:=', list_bodyType_test[6])
temp_df_Train = df[df['Train']==1]
temp=list(temp_df_Train[temp_df_Train['bodyType']==list_bodyType_test[6]].brand.unique())
print('Список брендов авто с типом кузова', list_bodyType_test[6], ':=', *temp)

Резюме: Авто с типом кузова лифтбек содержат марки авто которые мы собираемся оставить так что все ок, не нужно ничего добавлять к списку сверху

In [ ]:
# кроме этого в трейне нет некоторых авто БМВ с типом кузова
print('В трейне не оказалось авто БМВ с типом кузова:=', list_bodyType_test[9])
temp_df_Train = df[df['Train']==1]
temp=list(temp_df_Train[temp_df_Train['bodyType']==list_bodyType_test[9]].brand.unique())
print('Список брендов авто с типом кузова', list_bodyType_test[9], ':=', *temp)

In [ ]:
temp_df_Train[temp_df_Train['bodyType']==list_bodyType_test[9]]

У нас всего два таких авто БМВ с таким типов кузова, можно заметить по году выпуска, что это раритетные автомобили, поэтому им не с кем было сравнится по цене

In [ ]:
# оставляем 22 релевантных (21+BMW) бренд авто, остальные удаляем
df = df[df['brand'].isin(list_final_brand+['BMW'])]

***Резюме по brand*** - Проведен анализ средних и стандартных отклонений по ценам BMW по всем 11 типам кузова которые являются основным индикатором класса автомобиля. По результатам анализа выявлены 21 релевантные для моделирования цены бренды: AUDI CHERY CITROEN FORD KIA MAZDA OPEL RENAULT SKODA TOYOTA VOLKSWAGEN MERCEDES. Датасет урезан до этих брендов - осталось 22 вместе с BMW. Пропусков нет. 

In [ ]:
# записываем признак в список проанализированных признаков
old_len_train, EDA_done_cols = utils.result_EDA_feature('brand', df[df['Train']==1], df[df['Train']==0], 23, EDA_done_cols, old_len_train)

### 6.4 fuelType

In [ ]:
# вспомним что по результатам предварительного анализа в этой переменной было 6 уникальных значений в трейне и 4 в тесте
temp_df_Train = df[df['Train']==1]
temp_df_Test = df[df['Train']==0]
print(f'Список зачений по fuelType в трейне:= {list(temp_df_Train.fuelType.unique())}')
print(f'Список зачений по fuelType в тесте:= {list(temp_df_Test.fuelType.unique())}')

In [ ]:
# уберем газ так как автомобилей на газе нет в тесте
list_fuelType_test = temp_df_Test.fuelType.unique()
df = df[df['fuelType'].isin(list_fuelType_test)]

In [ ]:
# Посмотрим сколько элекромобилей в тесте
print('Распределение по кол-ву зачений по fuelType в трейне:= \n',temp_df_Train.fuelType.value_counts())
print('Распределение по кол-ву зачений по fuelType в тесте:= \n',temp_df_Test.fuelType.value_counts())

In [ ]:
# посмотрим на автомобили БМВ в трейне и в тесте
temp_df = temp_df_Train[(temp_df_Train['fuelType']=='электро')&(temp_df_Train['brand']=='BMW')]
print(f'Кол-во электрокаров BMW в трейне:= {len(temp_df)}')
display(temp_df)

display(temp_df_Test[(temp_df_Test['fuelType']=='электро')&(temp_df_Test['brand']=='BMW')])

In [ ]:
# посмотрим какие бренды в остальных (68-7=61) электрокаров трейна и как распределены по ним цены
temp_df = temp_df_Train[(temp_df_Train['fuelType']=='электро')] 
list_brand = utils.hbar_group_pivot_table(list_bodyType_test[3], 'price', temp_df, 2010, 2019, 1.1)

***Резюме по fuelType:-*** Категориальный признак. 4 значения ('газ' удалили). Электрокаров BMW в трейне - 7, в тесте - 1. Остальные 61 электрокар существенно ниже по цене, что может привести к снижению цены при моделировании БМВ. Кроме этого, визуальный осмотр значений электрокаров показал что стоит обратить внимание на обрабортку признака объем двигателя по этой группе. К сожалению нет фиксированной связи между можностью автомобиля и объемом двигателя, поэтому для моделирования важно сохранить оба признака, но по электрокарам нет значений и надо будет их как-то заполнить (может быть вывести из киловатов, но и они заполнены не везде)    

In [ ]:
# записываем признак в список проанализированных признаков
old_len_train, EDA_done_cols = utils.result_EDA_feature('fuelType', df[df['Train']==1], df[df['Train']==0], 23, EDA_done_cols, old_len_train)

### 6.5 engineDisplacement

In [ ]:
# обработаем значения с помощью регулярных выражений найдя значения объема в литра типа 2.0 и переведем их см3 
def engineDisplacement_to_float(row):
    row = str(row)
    volume = re.findall('\d\.\d', row)
    if volume == []:
        return None
    return int(float(volume[0])*1000)
# поле engineDisplacement заполненно не полностью именно мощностью, в отличии от поля name
# поэтому вытаскиваем мощность из поля name
df['engineDisplacement2'] = df['name'].apply(engineDisplacement_to_float)

In [ ]:
# посмотрим сколько пропусков
len(df[df['engineDisplacement2'].isna()])

In [ ]:
# подозрительно знакомое число видимо это электрокары, проверим 
df[df['engineDisplacement2'].isna()].fuelType.unique()

In [ ]:
# посмотрим что там внутри 
temp_df_Train = df[df['Train']==1]
temp_df = temp_df_Train[temp_df_Train['engineDisplacement2'].isna() & (temp_df_Train['brand']=='BMW')]
display(temp_df.head(2))
display(temp_df.describe())

In [ ]:
# чтож заполним пропуски аналогичными объемами двигателей автомобилей со следующими параметрами:
# 1 bodyType == хэтчбек 5 дв.
# 2 modelDate от 2013 до 2017
# 3 enginePower = около 170
# 4 price = 2.017469e+06	+ std(3.601556e+05) = (1657314, 2377624) 
# 5 brand == BMW
# 6 fuelType != 'электро'

temp_df = temp_df_Train[(temp_df_Train['bodyType']==list_bodyType_test[3]) & (temp_df_Train['brand']=='BMW') & (temp_df_Train['modelDate']>=2013) & (temp_df_Train['modelDate']<=2017)& (temp_df_Train['price']>=1657314) & (temp_df_Train['modelDate']<=2377624)& (temp_df_Train['enginePower']>=170-10) & (temp_df_Train['enginePower']<=170+10)& (temp_df_Train['fuelType']!='электро')]
display(temp_df.head(2))
display(temp_df.describe())

In [ ]:
# похоже что с такой низкой мощностью близкиеми оказались только гибриды, но 67 пропусков не так много - заполним мощность 700 см3, это на самом деле не объем двигателя гибрида действительно около 700 см3 мощностью 34 л.с, к которым добавляется 170 л.с электромотра в случае экземпляров из первых двух строк выше. И правильно было бы перевести их в усредненный объем двигателя соответсвующий 200 л.с., но с другой стороны все гибриды и электрокары будут с таким маленьким объемом и возможно он сможет найти закономерность и понять что гибриды и электрокары дороже своих аналогов 
df['engineDisplacement2'].fillna(700, inplace = True)

In [ ]:
# добавим мощность двигателя engineDisplacement2 в числовые признаки, а engineDisplacement удалим
cat_cols.remove('engineDisplacement') 
num_cols.append('engineDisplacement2')

In [ ]:
utils.describe_without_plots('engineDisplacement2', df[df['Train']==1].engineDisplacement2)

In [ ]:
# надо посмотреть где получились нули
df[df['engineDisplacement2']==0].head(5)

In [ ]:
# это два электрокара 2019 года выпуска с ценой выше в 2 раза средней по BMW
# удаляем
df = df[df['engineDisplacement2']!=0]

In [ ]:
utils.describe_without_plots('engineDisplacement2', df[df['Train']==1].engineDisplacement2)

In [ ]:
# а какой максимум по BMW в тесте
temp_df = df[df['Train']==0 & (df['brand']=='BMW')]
utils.describe_without_plots('engineDisplacement2', temp_df.engineDisplacement2)

In [ ]:
# какие авто с объемом двигателя более 6600 см3
temp_df= df[df['engineDisplacement2']>6600]
display(temp_df.head(3))
print(f'Кол-во авто с объемом двигателя более 6600 см3:= {len(temp_df)}')
print(f'Бренды авто с объемом двигателя более 6600 см3:= {list(temp_df.brand.unique())}')

In [ ]:
# удаляем эти 9
df = df[df['engineDisplacement2']<=6600]

In [ ]:
utils.four_plot_with_log2('engineDisplacement2', df[df['Train']==1])

In [ ]:
# добавим новый признак логарифм
df['engineDisplacement2_log'] = np.log(df['engineDisplacement2'] + 1)
num_cols.append('engineDisplacement2_log')

***Резюме по engineDisplacement:-*** Создан новый числовой признак объем двигателя в см3. После обработки пропуски оказались по электромобилям. Заполнили пропуски средними значениями по похожим параметрам - ими оказались гибриды с объемом 700 см3. Также удалили выбросы по порогу мин-макс теста по БМВ. Распределение скорее логнормальное - создали новый признак engineDisplacement2_log. 

In [ ]:
# записываем признак в список проанализированных признаков
old_len_train, EDA_done_cols = utils.result_EDA_feature('engineDisplacement', df[df['Train']==1], df[df['Train']==0], 23, EDA_done_cols, old_len_train)

### 6.6 numberOfDoors

In [ ]:
# переведем в int64 критерий numberOfDoors
df['numberOfDoors'] = df['numberOfDoors'].astype('int64')

In [ ]:
# посмотрим на зависимсть среднего значения цены от кол-ва дверей
df.groupby('numberOfDoors').mean().sort_values(by = 'price').price

In [ ]:
utils.describe_without_plots('numberOfDoors', df[df['Train']==1].numberOfDoors)

In [ ]:
utils.four_plot_with_log2('numberOfDoors', df[df['Train']==1])

In [ ]:
# фактически это категориальный признак с 4 значениями, но для корректной групповой обработки числовых признаков переведем его в числовые
cat_cols.remove('numberOfDoors')
num_cols.append('numberOfDoors')

In [ ]:
# записываем признак в список проанализированных признаков
old_len_train, EDA_done_cols = utils.result_EDA_feature('numberOfDoors', df[df['Train']==1], df[df['Train']==0], 23, EDA_done_cols, old_len_train)

### 6.7 mileage

In [ ]:
# переведем в int64 критерий numberOfDoors
df['mileage'] = df['mileage'].astype('int64')

In [ ]:
utils.describe_without_plots('mileage', df[df['Train']==1].mileage)

In [ ]:
utils.four_plot_with_log2('mileage', df[df['Train']==1])

In [ ]:
# посмотрим гистограммы на не новых автомобилях
utils.four_plot_with_log2('mileage', df[(df['Train']==1) & (df['mileage']>1000)])

In [ ]:
# посмотрим где проходят границы выбросов 
utils.borders_of_outliers('mileage',df[df['Train']==1], log = False)

In [ ]:
# посмотрим какие это марки с такими пробегами более 432000
df[(df['Train']==1) & (df['mileage']>432000)].brand.value_counts()

In [ ]:
# посмотрим статистику на тесте по пробегу
utils.describe_without_plots('mileage', df[df['Train']==0].mileage)

In [ ]:
# сколько авто в тесте за границей
df[(df['Train']==0) & (df['mileage']>432000)].brand.value_counts()

In [ ]:
# это полпроцента от объема теста удаляем все пробеги выше 432000 в трейне
df = df[((df['Train']==1) & (df['mileage']<=432000)) | ((df['Train']==0))]

***Резюме по mileage (пробег) *** - признак числовой. Удалили 446 выбросов с пробегом выше 432000 км из трейна. Пропусков нет.

In [ ]:
# записываем признак в список проанализированных признаков
old_len_train, EDA_done_cols = utils.result_EDA_feature('mileage', df[df['Train']==1], df[df['Train']==0], 23, EDA_done_cols, old_len_train)

### 6.8 Владельцы

In [ ]:
# напомню на этапе предварительного анализа мы не приводили этот признак к единому с тестом виду, поэтому надо будет полностью проанализировать значения и уже сразу понять в каком виде лучше их оставить для дальнейшего моделирования
# статистика по значениям в трейне
temp_df = df[df['Train']==1]
temp_df['Владельцы'].value_counts(normalize=True)

In [ ]:
# статистика по значениям в тесте
temp_df = df[df['Train']==0]
temp_df['Владельцы'].value_counts(normalize=True)

In [ ]:
# на этапе предварительного анализа были пропуски , проверим
len(df[df['Владельцы'].isna()])

In [ ]:
temp_df['Владельцы'].unique()

In [ ]:
# посмотрим статистику по пробегу по 2 владельцам 
df[(df['Владельцы']=='2\xa0владельца') & (df['mileage']>0)].mileage.describe()

In [ ]:
# посмотрим статистику по пробегу по 3 и более владельцам 
df[(df['Владельцы']=='3 или более') & (df['mileage']>0)].mileage.describe()

In [ ]:
# переведем в числовой формат 
# вытащим значение мощности перед первым пробелом'
df.loc[:, 'Владельцы'] = df['Владельцы'].apply(lambda x: int(x.split()[0]) if type(x)==str else None)

In [ ]:
# заполним Владельцев по следующему принципу
# пробег от 100000 км - 2 владельца
# пробег от 150000 км - 3 владельца
# если меньше 100000 км - 1 владелец
# и тут нас ждет фиаско потому что пробег по этим автомобилям 0
df.loc[df['Владельцы'].isna()].mileage.describe()

In [ ]:
# ну чтож посмотрим на пропуски в пробеге
df.loc[df['Владельцы'].isna()].head(3)

In [ ]:
# вроде бы это новые автомобили, проверим
df.loc[df['Владельцы'].isna()].productionDate.describe()

In [ ]:
# статистика по году производства в %
df[df['Владельцы'].isna()].productionDate.value_counts(normalize=True)

In [ ]:
# видно что это новые автомобили, поэтому заполним Владельцев нулями
df['Владельцы'].fillna(0.0, inplace=True)

In [ ]:
# переводим в int64
df['Владельцы'] = df['Владельцы'].astype('int64')

In [ ]:
utils.describe_without_plots('Владельцы', df[df['Train']==1].Владельцы)

In [ ]:
utils.four_plot_with_log2('Владельцы', df[df['Train']==1])

In [ ]:
# фактически это категориальный признак с 4 значениями, но для корректной групповой обработки числовых признаков переведем его в числовые
cat_cols.remove('Владельцы')
num_cols.append('Владельцы')

***Резюме по Владельцы*** - категориальный признак с 4 значениями. В ходе EDA выяснилось, что пропуски в новых автомобилях с годом производства 2020-2019 год, поэтому было добавлено еще одно значение 0. Теперь пропусков нет. Распределение с хвостом слева. 

In [ ]:
# записываем признак в список проанализированных признаков
old_len_train, EDA_done_cols = utils.result_EDA_feature('Владельцы', df[df['Train']==1], df[df['Train']==0], 23, EDA_done_cols, old_len_train)

### 6.9 price  
Посмотрим на стоимость выбранных нами авто из тренировочной выборки

In [ ]:
# сначала посмотрим на кол-во пропусков 
df[df['Train']==1]['price'].isna().sum()

In [ ]:
# затем как распределены пропуски по брендам
df[(df['Train']==1)&(df['price'].isna())]['brand'].value_counts()

In [ ]:
# Пропуски цены есть практически во всех моделях. Удаляем эти данные, т.к. их менее 1%, и даже если мы их будем заполнять ближними значениями, тратить смысл на подгонку целевых переменных нет
df = df[((df['Train']==1)&(df['price'].isna()==False)) | (df['Train']==0)]

In [ ]:
utils.describe_without_plots('price', df[df['Train']==1].price)

In [ ]:
utils.four_plot_with_log2('price', df[df['Train']==1])

In [ ]:
# распределение выглядит как логнормальное, посмотрим на выбросы и статистику мин-макс на BMW
# посмотрим где проходят границы выбросов 
utils.borders_of_outliers('price',df[df['Train']==1], log = True)

In [ ]:
# а что по БМВ?
utils.describe_without_plots('price', df[(df['Train']==1) & (df['brand']=='BMW')].price)

In [ ]:
# посмотрим сколько выбросов менее 60000 (мин БМВ) и более 	1.47812e+07 (макс БМВ)
len(df[(df['price']<60000) | (df['price']>1.47812e+07)])

In [ ]:
# не много взглянем на них
df[(df['price']<60000) | (df['price']>1.47812e+07)].head(3)

In [ ]:
# вроде как визуально старые авто проверим
df[(df['price']<60000) | (df['price']>1.47812e+07)].modelDate.value_counts(bins=5)

In [ ]:
# нет не старые совсем даже
# убедимся что БМВ среди них нет и будем удалять
df[(df['price']<60000) | (df['price']>1.47812e+07)].brand.value_counts()

In [ ]:
# удаляем выбросы по цене 
df = df[((df['Train']==1) & (df['price']>=60000) & (df['price']<1.47812e+07)) | (df['Train']==0)]

In [ ]:
# еще раз смотрим на гистограммы
utils.four_plot_with_log2('price', df[df['Train']==1])

In [ ]:
# ну теперь получилось очень красивое логнормальное распределение, хотя конечно необходимо было сначала проверить гипотезу о нормальном распределении, мы этот момент опустили из-за нехватки времени, просто создаем новый признак price_log логарифм от цены
df['price_log']=df['price'].apply(lambda x: np.log(x))

# добавим новый признак в список целевых, посмотрим что лучше обрабатывает медель потом при необходимости менее эффективный удалим
target_col.append('price_log')

***Резюме по price (стоимость)*** - целевая переменная. Удалили 220 пропусков из трейна. Удалили 253 выброса по цене снизу и сверху. Распределение скорее логнормальное и чистое без выросов. Добавили новый признак логарифм от прайса. Теперь пропусков нет.

In [ ]:
# записываем признак в список проанализированных признаков
old_len_train, EDA_done_cols = utils.result_EDA_feature('price', df[df['Train']==1], df[df['Train']==0], 23, EDA_done_cols, old_len_train)

### 6.10 modelDate

In [ ]:
utils.describe_without_plots('modelDate', df[df['Train']==1].modelDate)

In [ ]:
utils.describe_without_plots('modelDate', df[df['Train']==0].modelDate)

In [ ]:
# пропусков нет
# в тесте нет моделей 2020 нет моделей младше 1975 года 
# посмотрим сколько их
len(df[(df['Train']==1) & ((df['modelDate']<1975) | (df['modelDate']>2019))])

In [ ]:
# посмотрим на марки датой выпуска меньше 1975
df[(df['Train']==1) & (df['modelDate']<1975)].brand.value_counts()

In [ ]:
# их немного 38 удаляем точно, а что с марками датой выпуска 2020
df[(df['Train']==1) & (df['modelDate']==2020)].brand.value_counts()

In [ ]:
# можно было не смотреть марки 2020 года так как мы строим модель прогноза цен где таких моделей нет их нужно просто удалить по порогу и все
df = df[((df['Train']==1) & (df['modelDate']>=1975) & (df['modelDate']<=2019)) | (df['Train']==0)]

In [ ]:
# посмотрим гистограммы
utils.four_plot_with_log2('modelDate', df[df['Train']==1])

In [ ]:
# посмотрим на тест
utils.four_plot_with_log2('modelDate', df[df['Train']==0])

In [ ]:
# визуально можно заметить что основная часть автомобилей, что в тестовой, что в тренировочной выборке начинается с начала-середины 90х
# посмотрим еще срезы по кол-ву значений чтоы убедиться в этом
# статистика по значениям modelDate в трейне
temp_df = df[df['Train']==1]
temp_df['modelDate'].value_counts(bins=10)

In [ ]:
# статистика по значениям modelDate в тесте
temp_df = df[df['Train']==0]
temp_df['modelDate'].value_counts(bins=10)

***Резюме по modelDate*** - числовой признак, удалили выбросы 782 авто старше 1975 года и за 2020 год. Пропусков нет. Не удалось составить статистику завсисимости цены от года выпуска, даты производства и бренда авто, чтобы отделить раритетные автомобили и правильно прогнозировать по ним цену. Целевая метрика очень чувствительна к ошибкам - так если мы все автомобили угадаем на 100%, а в 50 автомобилях старше 1992 года сделаем ошибку в два раза на 500000 рублей при их стоимости примерно 500000 рублей, то  MAPE вырастет на = 1/3837\*(50\*100%) = 1.32%. С учетом того, что на лб разница между 10-м (9.76) и 20 местом (11.26) = 1.5%, то нужно стремиться научить модель предсказывать все автомобили достаточно хорошо. Мы не будем здесь расскрывать тему того, что целевая метрика обладает большими недостатками и плохо оценивает эффективность алгоритма прогнозирования. Мы сделаем это ниже.

In [ ]:
# записываем признак в список проанализированных признаков
old_len_train, EDA_done_cols = utils.result_EDA_feature('modelDate', df[df['Train']==1], df[df['Train']==0], 23, EDA_done_cols, old_len_train)

### 6.11 productionDate

In [ ]:
# посмотрим на трейн
utils.describe_without_plots('productionDate', df[df['Train']==1].productionDate)

In [ ]:
# теперь на тест
utils.describe_without_plots('productionDate', df[df['Train']==0].productionDate)

In [ ]:
# посмотрим сколько авто с датой производства 2020
len(df[(df['Train']==1) & (df['productionDate']>2019)])

In [ ]:
# такое большое кол-во это нормально так как датасет для соревнования формировался в феврале, то в тесте авто с датой производства 2020 нет - удалим их
# посмотрим сколько авто с датой производства менее 1981 в трейне
len(df[(df['Train']==1) & (df['productionDate']<1981)])

In [ ]:
# удалим выбросы по порогу
df = df[((df['Train']==1) & (df['productionDate']>=1981) & (df['productionDate']<=2019)) | (df['Train']==0)]

In [ ]:
utils.four_plot_with_log2('productionDate', df[df['Train']==1])

In [ ]:
utils.four_plot_with_log2('productionDate', df[df['Train']==0])

In [ ]:
# Посмотрим, как влияет год выпуска на распределение стоимости автомобиля
utils.four_plot_with_log2('price', df[(df['Train']==1)&(df['productionDate']<1990)])

In [ ]:
utils.four_plot_with_log2('price', df[(df['Train']==1)&(df['productionDate']>1990)])

In [ ]:
df[(df['Train']==1)&(df['productionDate']<1990)].price.describe()

In [ ]:
# Посмотрим на авто, стоимость которых заметно выше остальных:
utils.four_plot_with_log2('price', df[(df['Train']==1)&(df['productionDate']<1990)&(df['price']>300000)])

In [ ]:
# Добавим дополнительный признак 'intensity', который равен пробегу, поделенному на возраст авто, а также 'dateModelProdDiff', равный разнице между годом выпуска авто и годом начала производства модели
current_year = 2020
df['intensity']=df['mileage']/(current_year-df['productionDate'])
df['dateModelProdDiff']=df['productionDate']-df['modelDate']

# добавляем новые признаки
num_cols.append('intensity')
num_cols.append('dateModelProdDiff')

In [ ]:
# Посмотрим на стоимость еще внимательнее
df[(df['Train']==1)&(df['productionDate']<1990)&(df['price']>1000000)].describe()

In [ ]:
df[(df['Train']==1)&(df['productionDate']<1990)&(df['price']<1000000)].describe()

In [ ]:
# Пока непонятно как выделять раритеты (~10% от датасета), предположим, что на это влияет интенсивность использования. 75% для раритетов это около 4215 км/год, а для не раритетов 25% начинается с 5670 км/год. Попробуем разделить по границе 5000 км/год, добавим дополнительный признак 'rarity'
df['rarity']=(df['intensity']<5000)&(df['productionDate']<1990)

# добавляем новые признаки
bin_cols.append('rarity')

In [ ]:
# Посмотрим как меняется стоимость авто в зависимости от года выпуска
df_temp = df[(df['Train']==1)&(df['productionDate']<2020)&(df['rarity']==False)]
df_temp_bmw = df_temp[df['brand']=='BMW']
year = df_temp['productionDate'].values
price = df_temp['price'].values
year_bmw = df_temp_bmw['productionDate'].values
price_bmw = df_temp_bmw['price'].values
plt.figure(figsize=(20,10))
plt.scatter(year,price,c='b')
plt.scatter(year_bmw,price_bmw,c='r')

***Резюме*** Визуально характер стоимости делится на несколько участков:
- первые пять лет стоимость падает очень  быстро (видимо падает престиж)
- следующие 10 лет медленнее (авто перестает быть аттрибутом, становится инструментом)
- остальные посмотрим внимательнее

In [ ]:
df_temp = df[(df['Train']==1)&(df['productionDate']<2005)&(df['rarity']==False)]
df_temp_bmw = df_temp[df['brand']=='BMW']
year = df_temp['productionDate'].values
price = df_temp['price'].values
year_bmw = df_temp_bmw['productionDate'].values
price_bmw = df_temp_bmw['price'].values
plt.figure(figsize=(20,10))
plt.scatter(year,price,c='b')
plt.scatter(year_bmw,price_bmw,c='r')

***Резюме*** - Еще 15 лет стоимость как то держится, уменьшается количество машин, дальше уже начинаются непредсказуемые вещи, которые зависят от характера использования авто. Разделим датасет на несколько периодов:  
1. новее 2015  
2. 2015-2005  
3. 2005-1990  
4. старше 1990  

Чем старше авто, тем меньше в нем всяких электронных фич, тем больше мощность, посмотрим, будет ли этот признак иметь значение

In [ ]:
# добавляем 4 новых признака
df['pDate_more_2015']=df['productionDate']>=2015
df['pDate_more_2005']=(df['productionDate']>=2005)&(df['productionDate']<2015)
df['pDate_more_1990']=(df['productionDate']>=1990)&(df['productionDate']<2005)
df['pDate_less_1990']=(df['productionDate']<1990)

# добавляем новые признаки
bin_cols.append('pDate_more_2015')
bin_cols.append('pDate_more_2005')
bin_cols.append('pDate_more_1990')
bin_cols.append('pDate_less_1990')

***Резюме по productionDate*** - числовой признак,   
добавили 2 числовых признака: 
- intensity 
- dateModelProdDiff  

и 5 бинарных признаков:
- rarity
- pDate_more_2015
- pDate_more_2005
- pDate_more_1990
- pDate_less_1990

In [ ]:
# записываем признак в список проанализированных признаков
old_len_train, EDA_done_cols = utils.result_EDA_feature('productionDate', df[df['Train']==1], df[df['Train']==0], 23, EDA_done_cols, old_len_train)

### 6.12 vehicleConfiguration

In [ ]:
temp_df = df[df['Train']==1]
temp_df['vehicleConfiguration'].value_counts(normalize=True)

In [ ]:
# статистика по значениям в тесте
temp_df = df[df['Train']==0]
temp_df['vehicleConfiguration']

***Резюме по vehicleConfiguration*** - в нем есть все, что есть в других колонках, можно удалять

In [ ]:
# удаляем
cat_cols.remove('vehicleConfiguration')

In [ ]:
# записываем признак в список проанализированных признаков
old_len_train, EDA_done_cols = utils.result_EDA_feature('vehicleConfiguration', df[df['Train']==1], df[df['Train']==0], 23, EDA_done_cols, old_len_train)

### 6.13 vehicleTransmission

In [ ]:
# статистика по значениям в трейне
temp_df = df[df['Train']==1]
temp_df['vehicleTransmission'].value_counts()

In [ ]:
# статистика по значениям в тесте
temp_df = df[df['Train']==0]
temp_df['vehicleTransmission'].value_counts()

In [ ]:
# удалим вариатор
df = df[((df['Train']==1) & (df['vehicleTransmission']!='вариатор')) | (df['Train']==0)]

In [ ]:
# посмотрим на кол-во пропусков 
df[df['Train']==1]['vehicleTransmission'].isna().sum()

***Резюме по vehicleTransmission*** - категориальный признак, 4 значения, пропусков нет

In [ ]:
# записываем признак в список проанализированных признаков
old_len_train, EDA_done_cols = utils.result_EDA_feature('vehicleTransmission', df[df['Train']==1], df[df['Train']==0], 23, EDA_done_cols, old_len_train)

### 6.14 Комплектация

In [ ]:
# можно вспомнить предварительный анализ трейна с помощью PandasProffiling из которого следовало что этот признак не заполнен даже на половину (это значение "{'id': '0'}" было доминирующим). Посмотрим что изменилось
temp_df = df[df['Train']==1]
(temp_df['Комплектация']=="{'id': '0'}").sum()/len(temp_df)

In [ ]:
# более 75% процентов признака не заполнено в трейне, удаляем
cat_cols.remove('Комплектация')

***Резюме по Комплектация*** - более 75% процентов признака не заполнено в трейне, удаляем

In [ ]:
# записываем признак в список проанализированных признаков
old_len_train, EDA_done_cols = utils.result_EDA_feature('Комплектация', df[df['Train']==1], df[df['Train']==0], 23, EDA_done_cols, old_len_train)

### 6.15 description

In [ ]:
# Описание в трейне
temp_df = df[df['Train']==1]
temp_df['description'].iloc[2]

In [ ]:
# Описание в тесте
temp_df = df[df['Train']==0]
temp_df['description'].iloc[8]

***Резюме*** В тестовой выборке в данном признаке содержится в основном либо реклама салона, либо описание машины от владельца. В трейне иногда дополнительно содержится комплектация машины. Нужно очень много времени, чтобы проанализировать весь текст. Так что мы выделим некоторые особенности и отсеем их этапом ниже, если они будут давать плохой результат

In [ ]:
# заполним пропуски
df['description'] = df['description'].fillna('[]')

# запишем списки слов в описании в отдельный столбец
df['words_in_description'] = df['description'].apply(lambda x: [str(i).lower() for i in x.split()])

In [ ]:
# создаем новый признак кол-во слов в описании
df['count_words_d'] = df['description'].apply(lambda x: len(x.split()))

vectorizer = CountVectorizer()
text_feat = vectorizer.fit_transform(df['description'])

# создаем новые признаки кол-во среднее кол-во токенов и их сумма в описании
df['mean_c_w'] = text_feat.mean(axis=1)
df['sum_c_w'] = text_feat.sum(axis=1)

# удаляем 'description'
cat_cols.remove('description')

# добавляем 'count_words_d', 'mean_c_w', 'sum_c_w'
num_cols.append('count_words_d')
num_cols.append('mean_c_w')
num_cols.append('sum_c_w')

In [ ]:
print(*df['words_in_description'][:3], sep='===')

In [ ]:
# обработка слов в description
# выделяем словосочетания которые могут влиять на цену авто (дополнительный тюнинг или допопции при покупке нового авто) 
# защита картера  - crankcase protection
df['c_p_des1']= df['words_in_description'].apply(lambda x: 1 if ('защита' and 'картера') in x else 0)
bin_cols.append('c_p_des1')

# мультифункциональный руль - multifunction steering wheel
df['m_s_w_des2']= df['words_in_description'].apply(lambda x: 1 if ('мультифункциональный' and 'руль') in x else 0)
bin_cols.append('m_s_w_des2')

# датчики дождя и света - rain and light sensors
df['r_l_s_des3']= df['words_in_description'].apply(lambda x: 1 if ('датчики' and 'дождя' and 'света') in x else 0)
bin_cols.append('r_l_s_des3')

# АБС
df['abs_des4']= df['words_in_description'].apply(lambda x: 1 if ('антиблокировочная' and 'система') in x else 0)
bin_cols.append('abs_des4')

# круиз контроль - cruise control
df['c_c_des5']= df['words_in_description'].apply(lambda x: 1 if ('круиз-контроль') in x else 0)
bin_cols.append('c_c_des5')

# легкосплавные диски - alloy wheels
df['a_w_des6']= df['words_in_description'].apply(lambda x: 1 if ('легкосплавные' and 'диски') in x else 0)
bin_cols.append('a_w_des6')

# камера заднего вида - rear view camera
df['r_v_c_des7']= df['words_in_description'].apply(lambda x: 1 if ('камера' and 'видеокамера') in x else 0)
bin_cols.append('r_v_c_des7')

***Резюме по description*** - Выделили какие-то характерные признаки, вынесли их в отдельные столбцы. Вообще на анализ описания можно потратить достаточно много времени, поэтому пока остановимся

In [ ]:
# записываем признак в список проанализированных признаков
old_len_train, EDA_done_cols = utils.result_EDA_feature('description', df[df['Train']==1], df[df['Train']==0], 23, EDA_done_cols, old_len_train)

### 6.16  Привод

In [ ]:
temp_df = df[df['Train']==1]
temp_df['Привод'].value_counts()

In [ ]:
# статистика по значениям в тесте
temp_df = df[df['Train']==0]
temp_df['Привод'].value_counts()

In [ ]:
# посмотрим на кол-во пропусков 
df[df['Train']==1]['Привод'].isna().sum()

***Резюме по Привод*** - категориальный признак, 3 значения, пропусков нет

In [ ]:
# записываем признак в список проанализированных признаков
old_len_train, EDA_done_cols = utils.result_EDA_feature('Привод', df[df['Train']==1], df[df['Train']==0], 23, EDA_done_cols, old_len_train)

### 6.17  Состояние

In [ ]:
temp_df = df[df['Train']==0]
temp_df['Состояние'].value_counts()

In [ ]:
# удалим Состояние потому что не понятно как его обработать
cat_cols.remove('Состояние')

***Резюме по Состояние*** - мы не смогли заполнить этот признак во время парсинга, но как вид по тесту тоже только одно значение поэтому удаляем этот признак

In [ ]:
# записываем признак в список проанализированных признаков
old_len_train, EDA_done_cols = utils.result_EDA_feature('Состояние', df[df['Train']==1], df[df['Train']==0], 23, EDA_done_cols, old_len_train)

### 6.18 ПТС

In [ ]:
temp_df = df[df['Train']==1]
temp_df['ПТС'].value_counts()

In [ ]:
temp_df['ПТС'].isna().sum()

In [ ]:
temp_df[temp_df['ПТС'].isna()].head(3)

In [ ]:
# статистика по значениям в тесте
temp_df = df[df['Train']==0]
temp_df['ПТС'].value_counts()

In [ ]:
# Заполним отсутствующие значения вариантом "нет", т.к. это новые авто без ПТС
df['ПТС'].fillna('Нет', inplace = True)

***Резюме по ПТС*** - категориальный признак, 2 значения, пропусков нет

In [ ]:
# записываем признак в список проанализированных признаков
old_len_train, EDA_done_cols = utils.result_EDA_feature('ПТС', df[df['Train']==1], df[df['Train']==0], 23, EDA_done_cols, old_len_train)

### 6.18  Владение

In [ ]:
# Значения в трейне
temp_df = df[df['Train']==1]
temp_df['Владение'].value_counts()

In [ ]:
# Значения в тесте
temp_df = df[df['Train']==0]
temp_df['Владение']

***Резюме*** Мы видим, что формат записи и смысл у поля разный в разных датасетах: в тренировочном это английский язык и дата с момента начала владения, в тестовом - русский язык и время владения. Преобразуем эти значения в месяцы владения

In [ ]:
#заполняем пропуски значением nodata
df['Владение'] = df['Владение'].fillna('nodata')

def months_to_sent(months):
    if months == 1:
        return f'{months} месяц'
    elif 2 <= months <= 4:
        return f'{months} месяца'
    return f'{months} месяцев'
def years_to_sent(years):
    if 11 <= years <= 14 or 5 <= years%10 <= 9 or years%10 == 0:
        return f'{years} лет'
    elif years%10 == 1:
        return f'{years} год'
    elif 2 <= years%10 <= 4:
        return f'{years} годa'
def tenure(row):
    row = re.findall('\d+',row)
    if row != []:
        years = 2020 - (int(row[0])+1)
        months = 2 +(12 - int(row[1]))
        if years < 0:
            return months_to_sent(int(row[1]))
        elif years == 0 and months < 12:
            return months_to_sent(months)
        elif years >= 0 and months == 12:
            return years_to_sent(years + 1)
        elif years >= 0 and months > 12:
            return years_to_sent(years + 1)+' и '+months_to_sent(months - 12)
        elif years > 0 and months < 12:
            return years_to_sent(years)+' и '+months_to_sent(months)
        return None
    
df.loc[df['Train']==1,'Владение'] = df[df['Train']==1]['Владение'].apply(tenure)

In [ ]:
def num_of_months(row):
    if pd.notnull(row) and row!='nodata':
        list_ownership = row.split()
        if len(list_ownership) == 2:
            if list_ownership[1] in ['год', 'года', 'лет']:
                return int(list_ownership[0])*12
            return int(list_ownership[0])
        return int(list_ownership[0])*12 + int(list_ownership[3])

In [ ]:
df['num_of_month'] = df['Владение'].apply(num_of_months)
# добавляем кол-во месяцев владения
num_cols.append('num_of_month')
# удаляем владение 
cat_cols.remove('Владение')

In [ ]:
df['num_of_month'].value_counts(bins=50, normalize=True)[:10]

In [ ]:
df['num_of_month'].describe()

In [ ]:
df[df['num_of_month']>0].describe()

In [ ]:
# колво пропусков месяцев с нулевым пробегом
len(df[df['num_of_month'].isna() & (df['mileage']==0)])

In [ ]:
df[df['num_of_month']<3].head(3)

In [ ]:
df[(df['num_of_month']>0)&(df['num_of_month']<100)]['num_of_month'].hist(bins=100)

Время владения авто - это некий показатель надежности.
Если срок владения большой (более двух лет) - значит человек ездил на машине и ему было норм.
Если менее двух лет, то тоже норм, но ничего сказать не можем.
А вот если менее трех месяцев - то подозрительно, т.к. мало кто продает авто просто так: срочно или есть косяки, значит стоимость должна быть ниже.
Идея была в том, чтобы сравнить цены на одни и те же модели с длинным и коротким сроком владения. Но пока не смог придумать как это визуализировать. Но в любом случае, нужно этот признак попробовать как отдельный.
Сам по себе заполненный признак владения (если это не автосалон) - показатель надежности продажи. Если сможем вытащить из описания признак продажи от салона, то сделаем заполненный признак владения от частника отдельным признаком.
Пока же просто выделим отдельный категориальный признак Tenure:
-1 = срок владения менее 3 месяцев
0  = нет данных
1  = срок владения более трех месяцев

In [ ]:
# надо удалить владение, так как перед тем как над ним применять какиенибудь преобразования надо его заполнить, я заполнить его не представляется возможным так как много пропусков
num_cols.remove('num_of_month')

***Резюме по Владение*** - очень много пропусков. Сделали один отрицательный признак, если авто во владении менее 3 месяцев и два положительных, если более 3 месяцев и более года. Возможно, эти цифры надо подправить.

In [ ]:
# записываем признак в список проанализированных признаков
old_len_train, EDA_done_cols = utils.result_EDA_feature('Владение', df[df['Train']==1], df[df['Train']==0], 23, EDA_done_cols, old_len_train)

### 6.20  name

In [ ]:
# Значения в трейне
temp_df = df[df['Train']==1]
temp_df['name'].value_counts()

In [ ]:
# Значения в тесте
temp_df = df[df['Train']==0]
temp_df['name']

In [ ]:
# добавим что-то напоминающее модель в категориальные признаки 
df['model2'] = df['name'].apply(lambda x: str(x).split()[0])
cat_cols.append('model2')
# добавим что-то напоминающее характеристику модели в категориальные признаки 
df['model2_2'] = df['name'].apply(lambda x: str(x).split()[1])
cat_cols.append('model2_2')

***Резюме по name*** - выделили отдельные модели авто, признак категориальный. Добавили новый признак model2 - первое слово до пробела из name. name пока не удаляем из списка нужных приззнаков потому что он потребуется нам при обработке model

In [ ]:
# записываем признак в список проанализированных признаков
old_len_train, EDA_done_cols = utils.result_EDA_feature('name', df[df['Train']==1], df[df['Train']==0], 23, EDA_done_cols, old_len_train)

### 6.21 model

In [ ]:
# заполним значения модели по тесту , так как модель определяет класс автомобиля. А класс определяет цену. Сделаем это с помощью CatBoost

# приведем числовые критерии к int64 для CatBoost
for col in ['modelDate', 'productionDate', 'engineDisplacement2']:
    df[col] = df[col].astype('int64')

In [ ]:
# позволяет закрепить random_state во всей ячейке исполнения
np.random.seed(42)

le = LabelEncoder()

le.fit(df['bodyType'])
df['bodyType_'] = le.transform(df['bodyType'])

le.fit(df['name'])
df['name_'] = le.transform(df['name'])

le.fit(df['model2'])
df['model2_'] = le.transform(df['model2'])

le.fit(df['model2_2'])
df['model2_2_'] = le.transform(df['model2_2'])

le.fit(df['Привод'])
df['Привод_'] = le.transform(df['Привод'])

le.fit(df['fuelType'])
df['fuelType_'] = le.transform(df['fuelType'])

le.fit(df['vehicleTransmission'])
df['vehicleTransmission_'] = le.transform(df['vehicleTransmission'])

le.fit( df[df['Train']==1]['model'])
df.loc[df['Train']==1, 'model_'] = le.transform(df[df['Train']==1]['model'])

In [ ]:
# выведем соответсвия номеров в model_ и моделей БМВ
temp_train = df[(df['Train']==1) & (df['brand']=='BMW')]
list_model = list(temp_train.model_.unique())
for i in list_model:
    print(i, '==', temp_train[temp_train['model_']==i].model.values[0])

In [ ]:
# создаем список признаков для CatBoost
cols_for_CatBoost = ['bodyType_', 'name_', 'model2_', 'model2_2_', 'modelDate', 'engineDisplacement2', 'Привод_', 'fuelType_', 'enginePower', 'vehicleTransmission_']

In [ ]:
temp_train = df[(df['Train']==1) & (df['brand']=='BMW')]
temp_test = df[(df['Train']==0)]
train_data = temp_train[cols_for_CatBoost] # обучающая выборка
train_labels = temp_train['model_'] # метки принадлежности к классу
test_data = temp_test[cols_for_CatBoost] # тестовая выборка

model = CatBoostClassifier(iterations=50, learning_rate = 0.5, random_state=RANDOM_SEED) # классификатор
model.fit(train_data, train_labels) # обучение классификатора
prediction = model.predict(test_data) # передача тестовой выборки в модель
print(*list(prediction[:10])) # вывод результата "предсказания"

In [ ]:
# проверим первые 10 элементов теста
temp_test[['brand','bodyType', 'name', 'model', 'model2', 'modelDate', 'engineDisplacement2', 'Привод', 'fuelType', 'enginePower', 'vehicleTransmission']][:10]

In [ ]:
# выведем соотстсвующие номера после проверки
print(*[20, 13, 20, 416, 414, 416, 412, 410, 412, 20])

In [ ]:
print(*list(prediction[:10])) # вывод результата "предсказания"

авто с параметрами 30d 3.0d AT (249 л.с.) 4WD соответвуют X1, X3, X5, X6 и различие есть только по годам выпуска. Так что CatBoost успешно справился с задачей заполним модели по тесту

In [ ]:
df.loc[df['Train']==0, 'model_']=prediction

In [ ]:
# больше name нам не нужно удалим его
cat_cols.remove('name')
# также удалим model потому что теперьу нас есть числовой признак модели model_
cat_cols.remove('model')
# добавим новый признак
cat_cols.append('model_')

In [ ]:
# сохраним список номеров всех моделей BMW
list_all_num_model_BMW = df[df['brand']=='BMW'].model_.unique()

In [ ]:
# проверим пропуски
df['model_'].isna().sum()

***Резюме по model *** - категориальный признак, заполнен по тесту с помощью Catboost. Результат записан в новый признак model_. Пропусков нет. Всего 443 значения моделей в признаке.

In [ ]:
# записываем признак в список проанализированных признаков
old_len_train, EDA_done_cols = utils.result_EDA_feature('model', df[df['Train']==1], df[df['Train']==0], 23, EDA_done_cols, old_len_train)

### 6.22 start_date

In [ ]:
# удалим start_date потому что не понятно как его обработать
time_cols.remove('start_date')

In [ ]:
# записываем признак в список проанализированных признаков
old_len_train, EDA_done_cols = utils.result_EDA_feature('start_date', df[df['Train']==1], df[df['Train']==0], 23, EDA_done_cols, old_len_train)

### 6.23 color

In [ ]:
# Значения в трейне
temp_df = df[df['Train']==1]
temp_df['color'].value_counts()

In [ ]:
# Значения в трейне
temp_df = df[df['Train']==0]
temp_df['color'].value_counts()

***Резюме по color*** - категориальный признак, 16 значений, в тесте 15. пропусков нет

In [ ]:
# записываем признак в список проанализированных признаков
EDA_done_cols.append('color')
old_len_train, EDA_done_cols = utils.result_EDA_feature('', df[df['Train']==1], df[df['Train']==0], 23, EDA_done_cols, old_len_train)

### 7. Сохранение датасета после EDA 
---
датасет сохраняется на локальной машине, чтобы потом не было необходимости заново запускать ячейки ноутбука 

In [ ]:
#  этот раздел удален так как на каггле не требуется, оставлен только переход к df3

# сохраняем все переменные из списков
all_cols_df3 =cat_cols+num_cols+time_cols+servis_cols+bin_cols+target_col
# образаем исходный датасет только переменными, которые мы решили оставить
df3 = df.loc[:, all_cols_df3].copy()
utils.describe_without_plots_all_collumns(df3, short=True)

### 8. Перекрестный анализ между переменными и целевой переменной price
---
#### 8.1 Оценка корреляций

In [ ]:
utils.simple_heatmap('Матрица корреляции тренировочного датасета на числовых переменных',df3[df3['Train']==1], num_cols+target_col, 1.1, 1, 9)

***Выводы по матрице корелляции по всем маркам***: - c таргетом выше всего скоррелированы (по убыванию):
- modelDate (дата выхода модели)
- productionDate (дата производства)
- mileage (пробег)
- enginePower(мощность авто л.с.)
- engineDisplacement2_log (логарифм от объема двигателя)
- engineDisplacement2 (объем двигателя)
- Владельцы (кол-во)
- count_words_d, mean_c_w, sum_c_w (это данные из description кол-во слов всего, среднее и сумма не пересекающ.) 

и замыкают список интенсивность использования авто и разница в датах между выпуском и производством

In [ ]:
temp_df = df3[(df3['Train']==1) & (df3['brand']=='BMW')]
utils.simple_heatmap('Матрица корреляции тренировочного датасета на числовых переменных по  BMW',temp_df, num_cols+target_col, 1.1, 1, 9)

***Выводы по матрице корелляции ТОЛЬКО по BMW***: 
- c таргетом выше всего скоррелированы (по убыванию) :
- productionDate (дата производства)
- modelDate (дата выхода модели)
- mileage (пробег)
- Владельцы (кол-во)
- enginePower(мощность авто л.с.)
- engineDisplacement2_log (логарифм от объема двигателя), engineDisplacement2 (объем двигателя), count_words_d, mean_c_w, sum_c_w (это данные из description кол-во слов всего, среднее и сумма не пересекающ.) - примерно на одном уровне  
и замыкает список разница в датах между выпуском и производством  

Другие выводы:
- кореляция с таргетом enginePower(мощность авто л.с.) по BMW ниже чем по всем маркам
- кореляция с таргетом интенсивность использования авто и разница в датах между выпуском и производством выше на порядки, чем по всем маркам

***Выводы из матрицы корреляции ОБЩИЕ***:
- между собой высокоскоррелированны признаки:
  - modelDate (дата выхода модели), productionDate (дата производства), mileage (пробег). Владельцы (кол-во)
  - count_words_d, mean_c_w, sum_c_w (это данные из description кол-во слов всего, среднее и сумма не пересекающ.)
  - enginePower(мощность авто л.с.), engineDisplacement2_log (логарифм от объема двигателя) и engineDisplacement2 (объем двигателя)  
  
На текущем этапе мы пока не будем делать выводов об исключении признаков, а просто примем ко вниманию эту информацию

#### 8.2 Значимость непрерывных переменных по ANOVA F test

In [ ]:
temp_df = df3[df3['Train']==1]
imp_num = pd.Series(f_classif(temp_df[num_cols], temp_df['price_log'])[0], index = num_cols)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh', title='Значимость непрерывных переменных по ANOVA F test по всем маркам')

In [ ]:
temp_df = df3[(df3['Train']==1) & (df3['brand']=='BMW')]
imp_num = pd.Series(f_classif(temp_df[num_cols], temp_df['price_log'])[0], index = num_cols)
imp_num.sort_values(inplace = True)
imp_num.plot(kind = 'barh', title='Значимость непрерывных переменных по ANOVA F test по BMW')

***Выводы:*** 
- значимость тройки признаков - count_words_d, mean_c_w, sum_c_w (это данные из description кол-во слов всего, среднее и сумма не пересекающ.) снижается на BMW и приходит к уровню productionDate (дата производства) и modelDate (дата выхода модели
- значимость engineDisplacement2 (объем двигателя) падает на авто БМВ
- значимость интенсивность использования авто и разница в датах между выпуском не вырастает на БМВ, а за счет падения значимости остальных приходит в норму на БМВ

#### 8.3 Проверим статистическую значимость по всем не числовым признакам по тесту Стьюдента

In [ ]:
# переведем категориальные признаки в dummies переменные
# но сначала сохраним список переменных чтобы можно было сделать список добавленных
list_cols_bef_dumm = list(df3.columns)

# а также мы хотим преобразовать числовые признаки 'Владельцы' и 'engineDisplacement2'
# поэтому сохраним их дубликаты
arr_Владельцы = np.array(df3['Владельцы'])

# преобразуем переменные в дамми переменные
df3 = pd.get_dummies(df3, columns = ['bodyType', 'brand', 'color', 'fuelType', 'vehicleTransmission', 'Привод', 'Владельцы', 'ПТС', 'model_', 'engineDisplacement2'])

# вернем владельцев
df3['Владельцы'] = arr_Владельцы 

# теперь создадим список дамми переменных
list_cols_aft_dumm = list(df3.columns)
dumm_cols= list(set(list_cols_aft_dumm)-set(list_cols_bef_dumm ))

In [ ]:
print(f'Мы добавили:{len(dumm_cols)} dummies featuries')

In [ ]:
# удалим 'engineDisplacement2' из числовых признаков - они нам больше не понадобятся
num_cols.remove('engineDisplacement2')

# а также удалим категориальные признаки преобразованные в dummies их уже нет
drop_list_cols = ['bodyType', 'brand', 'color', 'fuelType', 'vehicleTransmission', 'Привод', 'ПТС', 'model_']
for col in drop_list_cols:
    cat_cols.remove(col)

In [ ]:
# Проверим, есть ли статистическая разница в распределении оценок по всем категориальным признакам, 
# с помощью теста Стьюдента. Проверим нулевую гипотезу о том, 
# что распределения price_log по различным параметрам неразличимы:
def get_stat_dif(d_column, d_df):
    cols = d_df.loc[:, d_column].value_counts().index[:]
    combinations_all = list(combinations(cols, 2))
    for comb in combinations_all:
        ttest = ttest_ind(d_df.loc[d_df.loc[:, d_column] == comb[0], 'price_log'].dropna(),
                          d_df.loc[d_df.loc[:, d_column] == comb[1], 'price_log'].dropna()).pvalue
        if  ttest<= 0.05/len(combinations_all): # Учли поправку Бонферони
            return(d_column)
            break

In [ ]:
stat_sign_diff_cols=[]

temp_df = df3[(df3['Train']==1)]
for col in bin_cols+dumm_cols:
    stat_sign_diff_cols.append(get_stat_dif(col,temp_df))
stat_sign_diff_cols = list(filter(None, stat_sign_diff_cols))

In [ ]:
not_stat_sign_diff_cols=list(set(bin_cols+dumm_cols) - set(stat_sign_diff_cols))
print(f'по тесту Стьюдента {len(not_stat_sign_diff_cols)} признаков из {len(bin_cols+dumm_cols)} бинарных и dummies признаков НЕ СОДЕРЖАТ СТАТИСТИЧЕСКИ ЗНАЧИМЫХ РАЗЛИЧИЙ с таргетом')
print('вот они:= ', *sorted(not_stat_sign_diff_cols))

***Резюме*** 
- brand - MINI можно удалить
- про bodyType_седан 2 дв мы помним там два раритетных авто, которые нам нужны (потому что что? правильно тип метрики в задаче такой, что нам нужно угадать всех особенно с низкими ценами иначе нас покарает MAPE)
- про объемы мы тоже помним - во впервых мы их обрезали снизу и сверху по БМВ. Но так как у гибридов минимальный объем двига 700 кубиков поэтому наверняка между гибридами и обычными авто остались объемы которые не делают погоды, но мы их пока трагать не будем
- проанализируем модели и избавимся от статистически не значимых не БМВ

In [ ]:
dumm_cols.remove('brand_MINI')
df3 = df3[df3['brand_MINI']!=1]

In [ ]:
check_list_model = [x for x in sorted(not_stat_sign_diff_cols) if 'model' in x]
print(f'всего моделей на проверку на удаление:= {len(check_list_model)}')

In [ ]:
# ранее мы сохранили список всех номеров моделей БМВ
print(*list_all_num_model_BMW)

In [ ]:
check_list_model_BMW = ['model__'+ str(x) for x in list_all_num_model_BMW]
print(*check_list_model_BMW)

In [ ]:
fin_drop_list_model = list(set(check_list_model)-set(check_list_model_BMW))
print(f'всего удаляем моделей после проверки:= {len(fin_drop_list_model)}')

In [ ]:
for col in fin_drop_list_model:
    dumm_cols.remove(col)
    df3 = df3[df3[col]!=1]

### 8.4 Проверим важность признаков с помощью случайного леса и отсечки по белому шуму
---
#### 8.4.1 Разбиваем датасет на тренировочный и валидационный (валидационный формирем похожим на тестовый по типу топлива и по датам выпуска)

In [ ]:
# сколько в трейне
len(df3[df3['Train']==1])

In [ ]:
# посмотрим сколько осталось авто БМВ в трейне
len(df3[(df3['Train']==1) & (df3['brand_BMW']==1)])

In [ ]:
# сколько в тесте
len(df3[df3['Train']==0])

Даже если бы мы делали прогноз цен по метрике MAPE 3837 авто из теста только по 4491 BMW из трейна, то этого было бы достаточно чтобы сделать оценку MAPE c точностью ниже 6.69 (1 место на лб). Это можно проверить с помощью дверительных интервалов. Так что ничего дополнительно парсить по БМВ с других сайтов не надо.

In [ ]:
# псомотрим еще раз на распределение авто бмв в тесте по дате марки
df3[(df3['Train']==0) & (df3['brand_BMW']==1)].productionDate.value_counts(bins=10, normalize=True)

In [ ]:
# Сделаем валидационную выборку из 800 авто
temp_ser = df3[(df3['Train']==0) & (df3['brand_BMW']==1)].productionDate.value_counts(bins=10, normalize=True)
temp_list = []
for interval in list(temp_ser.index):
    temp_list.append([interval, int(temp_ser[interval]*800)])
temp_list

In [ ]:
# пронумеруем трейн чтобы убрать из трейна то что попаедет в валидационнную выборку
df3.loc[df3['Train']==1,'id'] = np.array(range(1000000, 1000000+len(df3[df3['Train']==1])))

In [ ]:
# позволяет закрепить random_state во всей ячейке исполнения
np.random.seed(42)

df_val = pd.DataFrame()
i = 0
dict_id_val = {}
for interval in temp_list:
    df_sample = df3[(df3['Train']==1) & (df3['brand_BMW']==1) & (df3['productionDate'] > interval[0].left) & (df3['productionDate'] <= interval[0].right)].sample(interval[1]).copy()
    dict_id_val[i]=list(df_sample.id.unique())
    df_val = pd.concat([df_val, df_sample])
    i += 1
len(df_val)

In [ ]:
# визуальный контроль
df_val.head(2)

In [ ]:
df_train = df3[df3['Train']==1].copy()
# сделаем трейн без авто в валидационной выборке
for elem in dict_id_val:
    df_train = df_train[~df_train['id'].isin(dict_id_val[elem])]
len(df_train)

In [ ]:
# проверка сумма длин трайна и валидации = первоначальному трейну
len(df_train)+len(df_val) == len(df3[df3['Train']==1])

In [ ]:
# проверим распределение по типу топлива, чтобы убедиться что у нас в валидации доставточно гибридов и электрокаров
dumm_fuelType_cols = [x for x in dumm_cols if 'fuelType' in x]

print('Распределение fuelType в валидации:')
for col in dumm_fuelType_cols:
    print(col, ':=', int(len(df_val[df_val[col]==1])/len(df_val)*10000)/100, '%')

In [ ]:
# проверим распределение по типу топлива, чтобы убедиться что у нас в валидации доставточно гибридов и электрокаров
dumm_fuelType_cols = [x for x in dumm_cols if 'fuelType' in x]
temp_df = df3[df3['Train']==0]
print('Распределение fuelType в тесте:')
for col in dumm_fuelType_cols:
    print(col, ':=', int(len(temp_df[temp_df[col]==1])/len(temp_df)*100000)/1000, '%')

***Резюме*** Совсем уж так неполучится добавить, потому что добавление 1 авто к 795 это 0.1256%.У нас сейчас два электро и 2 гибрида. Поэтому удалим один электрокар, добавим 5 гибридов, чтобы отношение между гибридами и электрокарами 0.182/0.026 = 7 сохранилось. 

In [ ]:
# поммотрим какие электрокары в валидации
df_val[df_val['fuelType_электро']==1]

In [ ]:
# удаляем электрокар 2017 года так как этот интервал шире в тесте (см.выше)
# сначала добавляем в трейн
df_train = pd.concat([df_train,df_val[df_val['id']==1004755.0]])

# потом удаляем в валидации
df_val = df_val[df_val['id']!=1004755.0]

In [ ]:
# позволяет закрепить random_state во всей ячейке исполнения
np.random.seed(42)

# добавляем в валидацию 5 гибридов БМВ
df_sample = df_train[(df_train['brand_BMW']==1) & (df_train['fuelType_гибрид']==1)].sample(5).copy()
df_val = pd.concat([df_val, df_sample])

df_train = df_train[~df_train['id'].isin(list(df_sample.id.unique()))]

In [ ]:
# проверка сумма длин трайна и валидации = первоначальному трейну
len(df_train)+len(df_val) == len(df3[df3['Train']==1])

#### 8.4.2 Подготовка данных к машинному обучению

In [ ]:
# сохраняем все переменные из списков которые находятся в работе кроме дамми
all_cols_df4 =cat_cols+num_cols+servis_cols+bin_cols+target_col
# образаем исходный датасет только переменными, которые мы решили оставить
df4 = df_train.loc[:, all_cols_df4].copy()
utils.describe_without_plots_all_collumns(df4, short=True)

In [ ]:
df_train.head(2)

In [ ]:
df_val.head(2)

***Резюме*** пропусков нет, есть model2 и model2_2 не числовые поэтому их нужно удалить при построении модели

#### 8.4.3. Добавим белый шум

In [ ]:
w_n = np.random.normal(0, 1, size=len(df_train)+len(df_val))
df_train['w_n']=w_n[:len(df_train)]
df_val['w_n']=w_n[-len(df_val):]

#### 8.4.4 Строим модель случайного леса

In [ ]:
train = df_train.drop(['model2', 'model2_2', 'Train', 'id', 'price'], axis =1)
val = df_val.drop(['model2', 'model2_2', 'Train', 'id', 'price'], axis =1)

y_train = train.price_log.values            # наш таргет
X_train = train.drop(['price_log'], axis=1)

y_val = val.price_log.values            # наш таргет
X_val = val.drop(['price_log'], axis=1)

# проверяем
train.shape, X_train.shape, y_train.shape, val.shape, X_val.shape, y_val.shape

In [ ]:
model = RandomForestRegressor(random_state = RANDOM_SEED, n_jobs = -1, verbose = 1)
model.fit(X_train, y_train)
y_pred = model.predict(X_val)

#### 8.4.5 Оценка качества модели

In [ ]:
# в первый раз инициируем глобальную переменную с предыдущим скором
utils.last_pred = np.zeros((3,len(y_val)))

In [ ]:
utils.test_last_pred(y_val, y_pred, y_pred) if (utils.last_pred[0].max() == 0) else 0
utils.all_metrics_MAE_MPE_MAPE_WAPE_MSE_RMSE(np.exp(y_val), np.exp(y_pred))

***Резюме*** мы получили МАРЕ = 11.20, это в районе baseline, при этом коффициент детерминации очень близок 1 и средняя абсолютная ошибка 162522р, что достаточно хорошо с учетом средней цены на авто в валидации 1260395 р. RMSE повыше 269138 р, но тоже не критично. Не будем пока тут уделять внимание округлению, а перейдем к анализу важности признаков 

#### 8.4.6 Анализ переменных по важности (модель  Случайный лес)

In [ ]:
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X_train.columns)
feat_importances.nlargest(15).plot(kind='barh')

***Резюме*** белый шум на 11 месте из 583 переменных. Следовательно 572 (583-11) переменные можно удалить :). на самом деле попробуем поменять параметры и посмотреть как ведут себя метрики

#### 8.4.7 Тест различных моделей перед подбором оптимальных параметров (на параметрах по дефолту)

In [ ]:
# попробуем Случайный лес только на БМВ
train = df_train[df_train['brand_BMW']==1].drop(['model2', 'model2_2', 'Train', 'id', 'price'], axis =1)

y_train = train.price_log.values            # наш таргет
X_train = train.drop(['price_log'], axis=1)

# проверяем
train.shape, X_train.shape, y_train.shape, val.shape, X_val.shape, y_val.shape

In [ ]:
model = RandomForestRegressor(random_state = RANDOM_SEED, n_jobs = -1, verbose = 1)
model.fit(X_train, y_train)
y_pred = model.predict(X_val)
utils.test_last_pred(y_val, y_pred, y_pred) if (utils.last_pred[0].max() == 0) else 0
utils.all_metrics_MAE_MPE_MAPE_WAPE_MSE_RMSE(np.exp(y_val), np.exp(y_pred))

***Резюме*** - МАРЕ снизилась но не сильно, в целом есть небольшое снижение метрик ошибки, а коэффициент детерминации немного подрос

In [ ]:
# теперь Линейную регрессию на всем трейне
# Проверим LinearRegression
train = df_train.drop(['model2', 'model2_2', 'Train', 'id', 'price'], axis =1)
val = df_val.drop(['model2', 'model2_2', 'Train', 'id', 'price'], axis =1)

y_train = train.price_log.values            # наш таргет
X_train = train.drop(['price_log'], axis=1)

lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_val)
utils.test_last_pred(y_val, y_pred, y_pred) if (utils.last_pred[0].max() == 0) else 0
utils.all_metrics_MAE_MPE_MAPE_WAPE_MSE_RMSE(np.exp(y_val), np.exp(y_pred))

***Резюме*** - МАРЕ вырос на 2.44. Почти все метрики высоли. RMSE на 137165 р. И даже коэфф детерминации упал. Пока случайный лес показывает лучший результат.

In [ ]:
# попробуем ExtraTreeRegressor на всем трейне
etr = ExtraTreeRegressor(random_state = RANDOM_SEED)
etr.fit(X_train, y_train)
y_pred = etr.predict(X_val)
utils.test_last_pred(y_val, y_pred, y_pred) if (utils.last_pred[0].max() == 0) else 0
utils.all_metrics_MAE_MPE_MAPE_WAPE_MSE_RMSE(np.exp(y_val), np.exp(y_pred))

***Резюме*** - МАРЕ вырос еще на 3 %. Почти все метрики высоли, кроме MAE, MSE, RMSE на 137165 р. Коэфф детерминации немного поднялся. Несмотря на повышение процентных метрик, я считаю что процентные метрики нестабильны и плохо оценивают качество алгоритма. Поэтому тут ExtraTreeRegressor показал себя лучше Линейной регрессии, но пока случайный лес показывает лучший результат.

In [ ]:
# проверим градиентный бустинг на всем трейне
gbr = GradientBoostingRegressor(n_estimators=250)
gbr.fit(X_train, y_train)
y_pred = gbr.predict(X_val)
utils.test_last_pred(y_val, y_pred, y_pred) if (utils.last_pred[0].max() == 0) else 0
utils.all_metrics_MAE_MPE_MAPE_WAPE_MSE_RMSE(np.exp(y_val), np.exp(y_pred))

Резюме - МАРЕ снизился на 2.61 %. Почти все метрики снизились, кроме MPE. Коэфф детерминации немного поднялся. В целом градиентный бустинг показал себя лучше  ExtraTreeRegressor и Линейной регрессии, но пока случайный лес показывает лучший результат.

In [ ]:
# проверим BaggingRegressor вместе со случайным лесом на всем трейне
bgr_rf = BaggingRegressor(model, n_estimators=3, n_jobs=-1, random_state=RANDOM_SEED)
bgr_rf.fit(X_train, y_train)
y_pred = bgr_rf.predict(X_val)
utils.test_last_pred(y_val, y_pred, y_pred) if (utils.last_pred[0].max() == 0) else 0
utils.all_metrics_MAE_MPE_MAPE_WAPE_MSE_RMSE(np.exp(y_val), np.exp(y_pred))

***Резюме*** все метрики существенно снизились и коэффициент детерминации подрос. Запустим еще раз случайный лес, чтобы можно было сравнить прогресс

In [ ]:
model = RandomForestRegressor(random_state = RANDOM_SEED, n_jobs = -1, verbose = 1)
model.fit(X_train, y_train)
y_pred = model.predict(X_val)
utils.test_last_pred(y_val, y_pred, y_pred) if (utils.last_pred[0].max() == 0) else 0
utils.all_metrics_MAE_MPE_MAPE_WAPE_MSE_RMSE(np.exp(y_val), np.exp(y_pred))

***Резюме*** - Случайный лес без Бэгинга оказался лучше чем с Бэгингом. Надо учитавыть что это без настройки параметров.

In [ ]:
# попробуем стекинг Случайного леса и беггинг, пока без мета модели, просто возьмем  среднее
models = [RandomForestRegressor(random_state = RANDOM_SEED, n_jobs = -1, verbose = 1),
         BaggingRegressor(ExtraTreeRegressor(random_state=RANDOM_SEED), random_state=RANDOM_SEED)]

def stacking_model_predict(d_models, d_X_train, d_y_train, d_X_val):
    d_df = pd.DataFrame()
    for model_ in tqdm(d_models):
        model_.fit(d_X_train, d_y_train)
        y_pred = model_.predict(d_X_val)
        d_df[str(model_)[:6]] = np.exp(y_pred)                   
    return d_df

temp_df = stacking_model_predict(models, X_train, y_train, X_val)
temp_df['y_pred']=temp_df.mean(axis=1)

In [ ]:
y_pred = np.array(temp_df['y_pred'])
utils.test_last_pred(y_val, y_pred, y_pred) if (utils.last_pred[0].max() == 0) else 0
utils.all_metrics_MAE_MPE_MAPE_WAPE_MSE_RMSE(np.exp(y_val), y_pred)

***Резюме*** Стеккинг показал самый лучший результат. Но мы пока не касались вопросa округления. Проверим насколько это помогает.

In [ ]:
y_pred = np.round(np.array(temp_df['y_pred'])/1000,2)*1000
utils.test_last_pred(y_val, y_pred, y_pred) if (utils.last_pred[0].max() == 0) else 0
utils.all_metrics_MAE_MPE_MAPE_WAPE_MSE_RMSE(np.exp(y_val), y_pred)

In [ ]:
y_pred = np.round(np.array(temp_df['y_pred'])/100,2)*100
utils.test_last_pred(y_val, y_pred, y_pred) if (utils.last_pred[0].max() == 0) else 0
utils.all_metrics_MAE_MPE_MAPE_WAPE_MSE_RMSE(np.exp(y_val), y_pred)

In [ ]:
y_pred = np.round(np.array(temp_df['y_pred'])/10000,2)*10000
utils.test_last_pred(y_val, y_pred, y_pred) if (utils.last_pred[0].max() == 0) else 0
utils.all_metrics_MAE_MPE_MAPE_WAPE_MSE_RMSE(np.exp(y_val), y_pred)

#### 8.4.8 Проверим где мы больше всего ошибаемся на MAPE

In [ ]:
y_pred = np.round(np.array(temp_df['y_pred'])/10000,2)*10000
df_val['MAPE'] = np.round(np.abs((np.exp(y_val) - y_pred)/np.exp(y_val))*100,4)
df_val['y_pred'] = y_pred

In [ ]:
df_val['MAPE'].describe()

In [ ]:
df_val[df_val['MAPE']>35][['price', 'y_pred', 'MAPE', 'model2', 'modelDate','productionDate', 'enginePower', 'intensity', 'rarity', 'mileage', 'fuelType_бензин']]

In [ ]:
df_val[df_val['MAPE']>35][['price', 'y_pred', 'MAPE', 'model2', 'modelDate','productionDate', 'enginePower', 'intensity', 'rarity', 'mileage', 'fuelType_бензин']].describe()

***Резюме*** Мы ошибаемся более 35% MAPE на авто в среднем 2001 года, с мощностью бензинового двигателя 220 л.с. средней стоиомстью 783700 р.  

#### 8.4.9 Подбор параметров модели для Случайного леса

In [ ]:
# # закомментирован так как выполняется очень долго
# # позволяет закрепить random_state во всей ячейке исполнения
# np.random.seed(42)

# n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1500, num = 50)]
# max_features = ['auto', 'sqrt', 'log2']
# max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
# max_depth.append(None)
# min_samples_split = [2, 5, 10]
# min_samples_leaf = [1, 2, 4]
# bootstrap = [True, False]
# random_grid = {'n_estimators': n_estimators,
#                'max_features': max_features,
#                'max_depth': max_depth,
#                'min_samples_split': min_samples_split,
#                'min_samples_leaf': min_samples_leaf,
#                'bootstrap': bootstrap}

# rf = RandomForestRegressor()
# rf_random = RandomizedSearchCV(estimator = rf, 
#                                param_distributions = random_grid, 
#                                n_iter = 100, 
#                                cv = 3, 
#                                verbose=2, 
#                                random_state=RANDOM_SEED, 
#                                n_jobs = -1)
# rf_random.fit(X_train, y_train)

### 9. Построение итоговой модели 
---

In [ ]:
train = df3.query('Train==1').drop(['model2', 'model2_2', 'Train', 'id', 'price'], axis = 1)
test = df3.query('Train==0').drop(['model2', 'model2_2', 'Train', 'id', 'price'], axis = 1)

In [ ]:
y_train = train.price_log.values            # наш таргет
X_train = train.drop(['price_log'], axis=1)

X_test = test.drop(['price_log'], axis=1)

In [ ]:
models = [RandomForestRegressor(random_state = RANDOM_SEED, n_jobs = -1, verbose = 1),
         BaggingRegressor(ExtraTreeRegressor(random_state=RANDOM_SEED), random_state=RANDOM_SEED)]

def stacking_model_predict(d_models, d_X_train, d_y_train, d_X_test, d_df):
    for model_ in tqdm(d_models):
        model_.fit(d_X_train, d_y_train)
        y_pred = model_.predict(d_X_test)
        d_df[str(model_)[:6]] = np.round(np.exp(y_pred)/10000,2)*10000
    d_df['price'] = d_df.iloc[:,2:].mean(axis=1)    
    return 

stacking_model_predict(models, X_train, y_train, X_test, df_submit)

In [ ]:
df_submit.head(5)

In [ ]:
df_submit[['id', 'price']].to_csv(f'submission.csv', index=False)

### 11. Вишенка на торте или валерианка для слабонервных
---
После того как вы залили сабмишен на кагл не растраивайтесь, если вдруг вместо ожидаемых 9-11 MAPE получиться 20 и вы будете на 40 месте. Все нормально, дышите глубже. Все дело в том, что тест формировался в феврале 2020 года при курсе доллара 63 рубля. С тех времен много воды утекло и курс подрос. Поэтому для того, чтобы попасть выше вы должны подобрать коэффициент.  

Чтобы сделать подбор более оптимальным можно воспользоваться следующим WAPE<MAPE или русским языком взвешенная МАРЕ меньше МАРЕ. Посмотрите на формулу ошибки как на уравнение, где истинное значение это Х. Выбрав в сабмишене несколько срезов с одинаковыми ценами +-std и умножив их на како-нибудь коэффициент вы получите изменение скора. А далее по нему из уравнения можно с высокой точностью предсказать истинное значение. На мой взгляд 24 садмишена достаточно, чтобы оказаться в первой 10-ке.   
Главное не переживайте. Ваше эмоциональное здоровье - это наш приоритет! :)

**Резюме:**
score - 9.7602, получился на первоначальном датасете по марке BMW, умножением сабмишена на k=1.1

**Если ноутбук вам понравился прошу оценить стрелкой вверх**

Кроме этого вы можете оценить датасеты([датасет со всеми объявлениями авто_ру Москвы](https://www.kaggle.com/sokolovaleks/parsing-all-moscow-auto-ru-09-09-2020) и [датасет со всеми марками авто сайта авто_ру](https://www.kaggle.com/sokolovaleks/all-brands-and-models-for-auto-ru-09-09-2020) и [ноутбук для парсинга](https://www.kaggle.com/sokolovaleks/sf-dst-10-p1-parsing-almira-andrey-sokolov) по этой задаче 

**Спасибо за внимание**
Если остались вопросы и замечание - пишите